# Electra experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import ElectraModel, ElectraConfig, ElectraTokenizer, ElectraForSequenceClassification
from transformers import Trainer, TrainingArguments


In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_4.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Electra

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro', zero_division=0)
    precision = precision_score(y_true=labels, y_pred=pred, average='macro', zero_division=0)
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

20


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.956568,0.103093,0.062237,0.095000,0.030237
2,No log,2.829595,0.195876,0.107765,0.199643,0.123857
3,No log,2.691789,0.288660,0.200805,0.304762,0.218083
4,No log,2.586230,0.288660,0.204508,0.320952,0.228136
5,2.740400,2.452994,0.340206,0.231378,0.371548,0.258398
6,2.740400,2.371184,0.402062,0.357792,0.457262,0.352449
7,2.740400,2.248218,0.422680,0.349657,0.477024,0.373245
8,2.740400,2.183438,0.402062,0.370129,0.448690,0.363114
9,2.740400,2.090372,0.402062,0.399286,0.461190,0.386057
10,2.000800,2.071435,0.402062,0.390055,0.467857,0.376982


TrainOutput(global_step=1635, training_loss=1.9836382384693951, metrics={'train_runtime': 183.7421, 'train_samples_per_second': 71.105, 'train_steps_per_second': 8.898, 'total_flos': 384553275125760.0, 'train_loss': 1.9836382384693951, 'epoch': 15.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.90625   , -1.3212891 , -1.1796875 , ..., -1.9765625 ,
         1.5048828 ,  1.0332031 ],
       [-0.9926758 , -0.49072266, -0.05389404, ..., -1.8662109 ,
         0.7988281 ,  1.6152344 ],
       [-0.6665039 , -0.8857422 , -0.9785156 , ..., -1.0224609 ,
        -0.5053711 ,  1.4921875 ],
       ...,
       [-1.4570312 , -0.8364258 , -0.39013672, ..., -2.0996094 ,
         1.3251953 ,  1.5019531 ],
       [-0.55078125, -0.7988281 , -1.1308594 , ..., -0.30395508,
        -0.9267578 ,  0.36132812],
       [-1.5791016 , -0.8330078 , -0.5722656 , ..., -1.9912109 ,
         2.3027344 ,  0.7426758 ]], dtype=float32), label_ids=array([16,  3, 19, 16,  7, 18, 15,  4, 17,  2, 11,  5, 10,  0, 11, 16,  9,
       15,  9,  0, 10, 11,  5, 13,  8,  1, 10, 13,  0,  0,  8, 13,  8, 19,
       13,  2, 14,  4,  5,  0,  5,  7, 12, 10,  5, 16,  6, 16,  2,  1, 10,
        9, 12, 11,  6,  6,  4,  0,  3, 10,  4,  4, 19,  0, 13, 17, 13,  5,
        3,  7, 15, 19, 16,  2,  

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.990496,0.072165,0.010874,0.061667,0.017376
2,No log,2.978354,0.103093,0.011531,0.090000,0.020303
3,No log,2.906351,0.257732,0.149316,0.301310,0.176133
4,No log,2.779196,0.309278,0.296786,0.339643,0.264364
5,No log,2.617640,0.309278,0.209046,0.368095,0.245052
6,No log,2.521011,0.268041,0.220337,0.323333,0.212148
7,No log,2.293945,0.391753,0.358914,0.456905,0.360893
8,No log,2.152127,0.412371,0.485918,0.482619,0.395585
9,No log,2.040834,0.432990,0.432662,0.487262,0.413146
10,2.535200,1.951011,0.443299,0.474835,0.498571,0.433296


TrainOutput(global_step=660, training_loss=2.291194707697088, metrics={'train_runtime': 127.5575, 'train_samples_per_second': 81.94, 'train_steps_per_second': 5.174, 'total_flos': 307642620100608.0, 'train_loss': 2.291194707697088, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-0.42919922, -0.03207397, -0.6274414 , ...,  0.6557617 ,
         0.02593994, -0.48950195],
       [ 0.7861328 ,  0.90478516,  0.96777344, ..., -1.1660156 ,
        -0.8286133 , -0.19592285],
       [-1.3730469 , -0.9194336 , -1.4794922 , ...,  0.21240234,
        -0.23022461,  1.8388672 ],
       ...,
       [-1.0537109 , -0.28320312, -0.91064453, ...,  0.10693359,
         0.76220703,  1.1503906 ],
       [-1.4541016 , -1.0263672 , -1.6289062 , ...,  0.43286133,
        -0.28173828,  1.8330078 ],
       [-0.49902344,  0.5366211 ,  0.30566406, ..., -1.0175781 ,
         1.6210938 , -0.08215332]], dtype=float32), label_ids=array([16,  3, 19, 16,  7, 18, 15,  4, 17,  2, 11,  5, 10,  0, 11, 16,  9,
       15,  9,  0, 10, 11,  5, 13,  8,  1, 10, 13,  0,  0,  8, 13,  8, 19,
       13,  2, 14,  4,  5,  0,  5,  7, 12, 10,  5, 16,  6, 16,  2,  1, 10,
        9, 12, 11,  6,  6,  4,  0,  3, 10,  4,  4, 19,  0, 13, 17, 13,  5,
        3,  7, 15, 19, 16,  2,  